In [1]:
!pip install numpy scipy matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import wave
import scipy.io.wavfile as wav
from scipy.signal import resample, convolve
from itertools import zip_longest


In [3]:

# 读取wav文件
def read_wav(filename):
    Fs, data = wav.read(filename)
    return Fs, data


In [4]:

# 写入wav文件
def write_wav(filename, Fs, data):
    wav.write(filename, Fs, data.astype(np.int16))


In [5]:

# 重采样函数
def resample_audio(data, original_fs, new_fs):
    num_samples = int(len(data) * float(new_fs) / original_fs)
    return resample(data, num_samples)


### 修改处理函数，改用重叠相加法

- 我们每次会对数据进行分多个小段处理，也就是卷积
- 由于卷积会造成原数据端变长，我们需要将变长的小段存下来，加到下一次卷积结果的前端即可

In [6]:

# 处理单声道音频进行卷积
def process_audio(L,R, hrir_l, hrir_r, Fs, new_fs):
    # 重采样音频
    resampled_audio_L = resample_audio(L,Fs,new_fs)
    resampled_audio_R = resample_audio(R,Fs,new_fs)
    # 分别与左右HRIR卷积
    #out_l = convolve(resampled_audio_L, hrir_l)
    #out_r = convolve(resampled_audio_R, hrir_r)
    
    #设N=512为冲激串长度，M=256为分段长度，LenL=N+M-1=767结果长度
    N = 512
    M = 512
    LenL=N+M-1
    out_l=[]
    out_r=[]
    overlap_add_l=[0]*(LenL-M)
    overlap_add_r=[0]*(LenL-M)
    num_split=len(resampled_audio_L)//M
    for i in range(num_split):
        l,r=i*M,(i+1)*M
        res_l=convolve(resampled_audio_L[l:r],hrir_l)
        res_r=convolve(resampled_audio_R[l:r],hrir_r)
        out_l+=[a+b for a,b in zip_longest(res_l[:M],overlap_add_l,fillvalue=0)]
        out_r+=[a+b for a,b in zip_longest(res_r[:M],overlap_add_r,fillvalue=0)]
        overlap_add_l=res_l[M:-1]
        overlap_add_r=res_r[M:-1]

    # 合成立体声
    print(out_l[:10])
    out_stereo = np.column_stack((out_l, out_r))
    return out_stereo


In [8]:
L_path='full\\elev60\\L60e250a.wav'
R_path='full\\elev60\\R60e250a.wav'
# 读取HRIR数据
Fs_hrir, hrir_l = read_wav(L_path)
_, hrir_r = read_wav(R_path)
print(len(hrir_l))
hrir_l,hrir_r=hrir_l*0.00005,hrir_r*0.00005
print(max(hrir_l),max(hrir_r))


512
0.6887500000000001 0.29405000000000003


In [9]:

# 读取音频文件
Fs, audio_data = read_wav('test1.wav')
if len(audio_data[0])==2:
    audio_data_LL=[data[0] for data in audio_data]
    audio_data_RR=[data[1] for data in audio_data]
    print("双声道")
else:
    print('单声道')
print(max(audio_data_LL),max(audio_data_RR))

双声道
15951 15951


In [10]:
# 设置目标采样率
new_fs = 48000

# 处理音频，进行卷积并生成立体声
processed_audio = process_audio(audio_data_LL,audio_data_RR, hrir_l, hrir_r, Fs, new_fs)

# 将处理后的音频写入文件
write_wav('processed_audio.wav', new_fs, processed_audio)


[np.float64(-6.5431273508843756e-18), np.float64(-3.191592118931379e-17), np.float64(-6.470425935874549e-17), np.float64(-6.521316926381427e-17), np.float64(-1.2722747626719618e-17), np.float64(6.048757728817557e-17), np.float64(1.2933581730248117e-16), np.float64(1.4162235643914184e-16), np.float64(8.367932867631017e-17), np.float64(-1.7084832527309203e-17)]
